# IMPORTS

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

from functools import partial

import datashader as ds
from datashader import transfer_functions as tf

from datashader.utils import lnglat_to_meters as webm

from datashader.utils import export_image


from datashader.colors import colormap_select, Greys9, Hot, viridis, inferno, Elevation
from matplotlib.cm import hot
from colorcet import fire


from bokeh.plotting import figure, output_notebook, show
from datashader.bokeh_ext import InteractiveImage

from bokeh.tile_providers import STAMEN_TERRAIN, CARTODBPOSITRON, CARTODBPOSITRON_RETINA, STAMEN_TERRAIN_RETINA, STAMEN_TONER_LABELS, STAMEN_TONER, STAMEN_TONER_BACKGROUND


In [2]:
allaccidents = pd.read_csv('allaccidents.csv')

/Users/djurk/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
len(allaccidents)

1469788

In [4]:
allaccidents.head()

,Accident_Index,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,Junction_Detail,Junction_Control,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Year,Month,Hour,count,District_Name
0,2.01e+12,-2.775021,54.649766,3,3,1,2,02/01/2012,2,15:33,64,E10000006,3,66,Dual carriageway,70,NaN,NaN,-1,0,None within 50 metres,No physical crossing within 50 meters,Darkeness: No street lighting,Fine without high winds,Wet/Damp,None,None,2,Yes,E01019311,2012,1,15,1,Eden
1,2.01e+12,-2.897319,54.910354,3,3,1,1,03/01/2012,3,07:00,62,E10000006,1,6,Dual carriageway,70,NaN,NaN,-1,0,None within 50 metres,No physical crossing within 50 meters,Darkeness: No street lighting,Fine with high winds,Dry,None,None,2,Yes,E01019240,2012,1,7,1,Carlisle
2,2.01e+12,-2.650582,54.300442,3,2,1,3,02/01/2012,2,10:26,65,E10000006,1,6,Dual carriageway,70,NaN,NaN,-1,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Snowing with high winds,Snow,None,Other object in carriageway,2,Yes,E01019394,2012,1,10,1,South_Lakeland
3,2.01e+12,-2.711737,54.309084,3,3,1,1,04/01/2012,4,10:10,65,E10000006,6,0,Single carriageway,60,NaN,NaN,-1,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Raining without high winds,Wet/Damp,None,None,2,Yes,E01019394,2012,1,10,1,South_Lakeland
4,2.01e+12,-2.856827,54.641216,3,3,3,1,06/01/2012,6,18:30,64,E10000006,3,66,Dual carriageway,70,NaN,NaN,-1,0,None within 50 metres,No physical crossing within 50 meters,Darkeness: No street lighting,Raining with high winds,Wet/Damp,None,Involvement with previous accident,2,Yes,E01019339,2012,1,18,1,Eden


In [5]:
allaccidents.shape

(1469788, 35)

In [6]:
allaccidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1469788 entries, 0 to 1469787
Data columns (total 35 columns):
Accident_Index                                 1469788 non-null object
Longitude                                      1469788 non-null float64
Latitude                                       1469788 non-null float64
Police_Force                                   1469788 non-null int64
Accident_Severity                              1469788 non-null int64
Number_of_Vehicles                             1469788 non-null int64
Number_of_Casualties                           1469788 non-null int64
Date                                           1469788 non-null object
Day_of_Week                                    1469788 non-null int64
Time                                           1469788 non-null object
Local_Authority_(District)                     1469788 non-null int64
Local_Authority_(Highway)                      1469788 non-null object
1st_Road_Class                         

In [7]:
vizdf = allaccidents.drop(['Accident_Index','Police_Force','Junction_Detail','Junction_Control',
                           'LSOA_of_Accident_Location','1st_Road_Number','2nd_Road_Number'], axis = 1)

In [8]:
vizdf.head()

,Longitude,Latitude,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,Road_Type,Speed_limit,2nd_Road_Class,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,Year,Month,Hour,count,District_Name
0,-2.775021,54.649766,3,1,2,02/01/2012,2,15:33,64,E10000006,3,Dual carriageway,70,-1,None within 50 metres,No physical crossing within 50 meters,Darkeness: No street lighting,Fine without high winds,Wet/Damp,None,None,2,Yes,2012,1,15,1,Eden
1,-2.897319,54.910354,3,1,1,03/01/2012,3,07:00,62,E10000006,1,Dual carriageway,70,-1,None within 50 metres,No physical crossing within 50 meters,Darkeness: No street lighting,Fine with high winds,Dry,None,None,2,Yes,2012,1,7,1,Carlisle
2,-2.650582,54.300442,2,1,3,02/01/2012,2,10:26,65,E10000006,1,Dual carriageway,70,-1,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Snowing with high winds,Snow,None,Other object in carriageway,2,Yes,2012,1,10,1,South_Lakeland
3,-2.711737,54.309084,3,1,1,04/01/2012,4,10:10,65,E10000006,6,Single carriageway,60,-1,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Raining without high winds,Wet/Damp,None,None,2,Yes,2012,1,10,1,South_Lakeland
4,-2.856827,54.641216,3,3,1,06/01/2012,6,18:30,64,E10000006,3,Dual carriageway,70,-1,None within 50 metres,No physical crossing within 50 meters,Darkeness: No street lighting,Raining with high winds,Wet/Damp,None,Involvement with previous accident,2,Yes,2012,1,18,1,Eden


In [9]:
#Converting to categorical variables
vizdf['Road_Type'] = vizdf['Road_Type'].astype('category')
vizdf['Light_Conditions'] = vizdf['Light_Conditions'].astype('category')
vizdf['Weather_Conditions'] = vizdf['Weather_Conditions'].astype('category')
vizdf['Road_Surface_Conditions'] = vizdf['Road_Surface_Conditions'].astype('category')
vizdf['Accident_Severity'] = vizdf['Accident_Severity'].astype('category')

In [10]:
vizdf['Road_Surface_Conditions'].value_counts()

Dry                          1011379
Wet/Damp                      413401
Frost/Ice                      30658
Snow                           10342
Flood (Over 3cm of water)       2068
Name: Road_Surface_Conditions, dtype: int64

In [11]:
vizdf['Road_Type'].value_counts()

Single carriageway    1100515
Dual carriageway       216865
Roundabout              98508
One way street          30329
Slip road               15357
Unknown                  8214
Name: Road_Type, dtype: int64

In [12]:
#Converting Lat/Long values to Web Mercador values (standard for web visualization)
webx, weby = [x for x in list(webm(vizdf.loc[:,'Longitude'], vizdf.loc[:,'Latitude']))]

In [13]:
vizdf = vizdf.assign(webm_x = webx).copy()
vizdf = vizdf.assign(webm_y = weby).copy()

In [14]:
vizdf.head()

,Longitude,Latitude,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,Road_Type,Speed_limit,2nd_Road_Class,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,Year,Month,Hour,count,District_Name,webm_x,webm_y
0,-2.775021,54.649766,3,1,2,02/01/2012,2,15:33,64,E10000006,3,Dual carriageway,70,-1,None within 50 metres,No physical crossing within 50 meters,Darkeness: No street lighting,Fine without high winds,Wet/Damp,None,None,2,Yes,2012,1,15,1,Eden,-308913.924661,7.294187e+06
1,-2.897319,54.910354,3,1,1,03/01/2012,3,07:00,62,E10000006,1,Dual carriageway,70,-1,None within 50 metres,No physical crossing within 50 meters,Darkeness: No street lighting,Fine with high winds,Dry,None,None,2,Yes,2012,1,7,1,Carlisle,-322528.075746,7.344487e+06
2,-2.650582,54.300442,2,1,3,02/01/2012,2,10:26,65,E10000006,1,Dual carriageway,70,-1,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Snowing with high winds,Snow,None,Other object in carriageway,2,Yes,2012,1,10,1,South_Lakeland,-295061.438546,7.227263e+06
3,-2.711737,54.309084,3,1,1,04/01/2012,4,10:10,65,E10000006,6,Single carriageway,60,-1,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Raining without high winds,Wet/Damp,None,None,2,Yes,2012,1,10,1,South_Lakeland,-301869.182005,7.228912e+06
4,-2.856827,54.641216,3,3,1,06/01/2012,6,18:30,64,E10000006,3,Dual carriageway,70,-1,None within 50 metres,No physical crossing within 50 meters,Darkeness: No street lighting,Raining with high winds,Wet/Damp,None,Involvement with previous accident,2,Yes,2012,1,18,1,Eden,-318020.526924,7.292543e+06


In [15]:
#Establishing bounding box for the entire area
bboxes = {
  "gb": ((-15.381, 7.251), (48.749, 61.502)),
 }

In [16]:
PLOT_WIDTH = 750
def get_plot_size(x_range, y_range, plot_width=PLOT_WIDTH):
    x_range_dist = x_range[1] - x_range[0]
    y_range_dist = y_range[1] - y_range[0]
    plot_height = int(y_range_dist * plot_width /x_range_dist)
    return (plot_width, plot_height)

In [17]:
def get_plot_params(bbox, plot_width=PLOT_WIDTH):
    x_range, y_range = webm(*bbox)
    plot_width, plot_height = get_plot_size(x_range, y_range, plot_width=plot_width)
    return x_range, y_range, plot_width, plot_height

In [18]:
background = 'black'

In [19]:
export = partial(export_image, background=background, export_path="export")

# utility function: color map
cm = partial(colormap_select, reverse=(background!="black"))

In [20]:
x_range, y_range, plot_width, plot_height = get_plot_params(bboxes["gb"])
cvs = ds.Canvas(plot_width, plot_height, x_range, y_range)
agg = cvs.points(vizdf, 'webm_x', 'webm_y')

In [21]:
output_notebook()

Loading BokehJS ...

In [26]:
#Setting up map base
def base_plot(x_range, y_range, plot_width, plot_height,
              tools='pan, wheel_zoom, reset', **plot_args):
      
    p = figure(
      x_range=x_range,
      y_range=y_range,
      plot_width=plot_width,
      plot_height=plot_height,
      tools=tools,
      outline_line_color=None,
      min_border=0,
      min_border_left=0,
      min_border_right=0,
      min_border_top=0,
      min_border_bottom=0,
      **plot_args)
    
    p.axis.visible = False 
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    return p

In [23]:
def agg_by_road_type(x_range, y_range, w, h):
    cvs = ds.Canvas(x_range=x_range, y_range=y_range, plot_width=w, plot_height=h)
    agg = cvs.points(vizdf, 'webm_x', 'webm_y',  ds.count_cat('Road_Type'))
    img = tf.shade(agg, cmap=Hot, color_key=color_key, how='eq_hist')
    return tf.dynspread(img, threshold=0.5, max_px=4)

In [24]:
color_key = {
  'Dual carriageway':'red',
  'One way street':'yellow',
  'Roundabout':'green',
  'Single carriageway':'royalblue',
  'Slip road':'purple',
  'Unknown':'black'
}

In [27]:
#Visual map of traffic accidents with different street types as different colors
p = base_plot(*get_plot_params(bboxes["gb"]), background_fill_color=background)
p.add_tile(CARTODBPOSITRON_RETINA)  
InteractiveImage(p, agg_by_road_type)